<a href="https://colab.research.google.com/github/ameliachoi/coursera-IBM-applied-data-science-capstone/blob/master/coursera_applied_data_science_capstone_project_jinchoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[Coursera] Applied Data Science Capstone**

# The Battle of Neighborhoods
## Seoul, South Korea - Restaurant "Likes" Prediction Using Foursquare API and Machine Learning

---

### 1. Introduction

In [ ]:
! pip install geocoder
! pip install folium

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

from urllib.request import urlopen
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import pylab as pl

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score, confusion_matrix, classification_report, log_loss, r2_score, mean_squared_error
import itertools

import plotly
import plotly.graph_objects as go
import plotly.express as px

print('Libraries imported.')


Libraries imported.


---

### 2. Data

#### 2.1. Data Scraping and Cleaning

In [ ]:
# define a function to get the geocodes that is lats and lons using geopy
address = 'Seoul, Republic of Korea'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Seoul, Republic of Korea are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Seoul, Republic of Korea are 37.5666791, 126.9782914.


In [ ]:
CLIENT_ID = 'AQLM5MDGGIQWD5VD4LJS3XXSYEJJS34SPWSFWAR1THI1GZMZ' # your Foursquare ID
CLIENT_SECRET = 'EOW1J3GXQVU2JDWGOD3KG0YV3VEG0F1JQS3CAGP1V20QMBO0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

print(url)

Your credentails:
CLIENT_ID: AQLM5MDGGIQWD5VD4LJS3XXSYEJJS34SPWSFWAR1THI1GZMZ
CLIENT_SECRET:EOW1J3GXQVU2JDWGOD3KG0YV3VEG0F1JQS3CAGP1V20QMBO0
https://api.foursquare.com/v2/venues/explore?&client_id=AQLM5MDGGIQWD5VD4LJS3XXSYEJJS34SPWSFWAR1THI1GZMZ&client_secret=EOW1J3GXQVU2JDWGOD3KG0YV3VEG0F1JQS3CAGP1V20QMBO0&v=20180605&ll=37.5666791,126.9782914&radius=1000&limit=100


In [ ]:
# scrape the data from the generated URL
result = requests.get(url).json()
result.head()

{'meta': {'code': 200, 'requestId': '5f0d593620a61b42e0b8a578'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ba1a9adf964a5209bc637e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/korean_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d113941735',
         'name': 'Korean Restaurant',
         'pluralName': 'Korean Restaurants',
         'primary': True,
         'shortName': 'Korean'}],
       'id': '4ba1a9adf964a5209bc637e3',
       'location': {'address': '중구 을지로1길 38',
        'cc': 'KR',
        'city': '서울특별시',
        'country': '대한민국',
        'distance': 183,
        'formattedAddress': ['중구 을지로1길 38',
         '명동',
         '서울특별시',
         '서울특별시',
         '04522',
         '대한민국'],
        'labeledLatLngs': [{'label': 'display',
  

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
  try:
    categories_list = row['categories']
  except:
    categories_list = row['venue.categories']
  
  if len(categories_list) == 0:
    return None
  else:
    return categories_list[0]['name']

In [ ]:
venues = result['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name',
                   'venue.categories',
                   'venue.location.lat',
                   'venue.location.lng',
                   'venue.id']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by FourSquare.'.format(nearby_venues.shape[0]))

100 venues were returned by FourSquare.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [ ]:
nearby_venues.head()

,name,categories,lat,lng,id
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3
1,Seoul Plaza (서울광장),Pedestrian Plaza,37.565475,126.977937,4baf0f0bf964a52021e93be3
2,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5
3,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254
4,The Plaza Hotel (더 플라자),Hotel,37.564621,126.978060,4b058782f964a520799622e3


In [ ]:
nearby_venues['categories'].unique()

array(['Korean Restaurant', 'Pedestrian Plaza', 'Chocolate Shop', 'Hotel',
       'Historic Site', 'Seafood Restaurant', 'Lounge', 'Café',
       'Sushi Restaurant', 'Palace', 'Pub', 'Bookstore',
       'Chinese Restaurant', 'Buffet', 'Gukbap Restaurant', 'Plaza',
       'Bistro', 'Burger Joint', 'Dessert Shop', 'Japanese Restaurant',
       'Hotel Bar', 'Coffee Shop', 'Tourist Information Center',
       'Tea Room', 'Bagel Shop', 'Steakhouse', 'Art Museum', 'Theater',
       'Dog Run', 'Scenic Lookout', 'BBQ Joint', 'Optical Shop',
       'Bossam/Jokbal Restaurant', 'Beer Garden', 'Indie Movie Theater',
       'Souvenir Shop', 'Exhibit', 'Food Court', 'Mexican Restaurant',
       'Vietnamese Restaurant', 'Performing Arts Venue', 'Noodle House',
       'Bakery', 'Italian Restaurant', 'Cosmetics Shop',
       'Indian Restaurant'], dtype=object)

In [ ]:
resturant_list = ['Korean Restaurant', 'Chocolate Shop', 
                'Seafood Restaurant', 'Café', 'Sushi Restaurant', 
                'Pub', 'Chinese Restaurant', 'Buffet', 'Gukbap Restaurant', 
                'Bistro', 'Burger Joint', 'Dessert Shop', 'Japanese Restaurant',
                'Hotel Bar', 'Coffee Shop', 'Tea Room', 'Bagel Shop', 'Steakhouse', 
                'BBQ Joint', 'Bossam/Jokbal Restaurant', 'Beer Garden', 
                'Food Court', 'Mexican Restaurant','Vietnamese Restaurant', 'Noodle House',
                'Bakery', 'Italian Restaurant', 'Indian Restaurant']

In [ ]:
nearby_venues = nearby_venues[nearby_venues['categories'].isin(resturant_list)]
nearby_venues['categories'].unique()

array(['Korean Restaurant', 'Chocolate Shop', 'Seafood Restaurant',
       'Café', 'Sushi Restaurant', 'Pub', 'Chinese Restaurant', 'Buffet',
       'Gukbap Restaurant', 'Bistro', 'Burger Joint', 'Dessert Shop',
       'Japanese Restaurant', 'Hotel Bar', 'Coffee Shop', 'Tea Room',
       'Bagel Shop', 'Steakhouse', 'BBQ Joint',
       'Bossam/Jokbal Restaurant', 'Beer Garden', 'Food Court',
       'Mexican Restaurant', 'Vietnamese Restaurant', 'Noodle House',
       'Bakery', 'Italian Restaurant', 'Indian Restaurant'], dtype=object)

In [ ]:
# set up to pull the 'likes' from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
  venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
  url_list.append(venue_url)

for link in url_list:
  result = requests.get(link).json()
  likes = result['response']['likes']['count']
  like_list.append(likes)
print(like_list)

nearby_venues['likes'] = like_list
nearby_venues.head()

[192, 41, 57, 46, 13, 114, 50, 16, 81, 74, 109, 42, 71, 19, 79, 50, 72, 96, 46, 139, 41, 9, 30, 78, 98, 22, 22, 16, 46, 137, 92, 10, 7, 12, 20, 14, 15, 23, 16, 222, 7, 9, 31, 12, 40, 118, 116, 10, 58, 142, 13, 27, 11, 47, 12, 10, 199, 17, 15, 8, 37, 38]


,name,categories,lat,lng,id,likes
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192
2,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41
3,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13


In [ ]:
# rename is something more appropriate
raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,likes
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192
2,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41
3,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13


#### 2.2 Data Preparation

In [ ]:
# grouping specific categories
raw_dataset['categories'].unique()

array(['Korean Restaurant', 'Chocolate Shop', 'Seafood Restaurant',
       'Café', 'Sushi Restaurant', 'Pub', 'Chinese Restaurant', 'Buffet',
       'Gukbap Restaurant', 'Bistro', 'Burger Joint', 'Dessert Shop',
       'Japanese Restaurant', 'Hotel Bar', 'Coffee Shop', 'Tea Room',
       'Bagel Shop', 'Steakhouse', 'BBQ Joint',
       'Bossam/Jokbal Restaurant', 'Beer Garden', 'Food Court',
       'Mexican Restaurant', 'Vietnamese Restaurant', 'Noodle House',
       'Bakery', 'Italian Restaurant', 'Indian Restaurant'], dtype=object)

In [ ]:
Korean = ['Korean Restaurant', 'Seafood Restaurant',
          'Gukbap Restaurant', 'Bossam/Jokbal Restaurant']

Asian = ['Sushi Restaurant', 'Chinese Restaurant',
         'Japanese Restaurant', 'Vietnamese Restaurant', 'Indian Restaurant']

Europe = ['Bistro', 'Italian Restaurant']

American = ['Burger Joint', 'Steakhouse', 'BBQ Joint', 'Mexican Restaurant']

Bar = ['Pub', 'Hotel Bar', 'Beer Garden']

Casual = ['Chocolate Shop', 'Café', 'Buffet',
          'Dessert Shop', 'Coffee Shop', 'Tea Room',
          'Bagel Shop', 'Food Court', 'Noodle House', 'Bakery']

In [ ]:
def conditions(s):
  if s['categories'] in Korean:
    return 'Korean'
  elif s['categories'] in Asian:
    return 'Asian'
  elif s['categories'] in Europe:
    return 'Europe'
  elif s['categories'] in Bar:
    return 'Bar'
  elif s['categories'] in American:
    return 'American'
  else:
    return 'Casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,likes,categories_classified
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192,Korean
2,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41,Korean
3,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57,Casual
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46,Korean
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13,Casual
10,Sushi Cho (스시 조),Sushi Restaurant,37.564491,126.979743,4f82efddd4f208b5bd478f0e,114,Asian
14,Wolhyang (월향),Pub,37.567916,126.975665,566033b1498e93c84b700d64,50,Bar
16,열빈,Chinese Restaurant,37.567522,126.976187,4f969714e4b07e54f0d8cdb9,16,Asian
17,La Seine (라세느),Buffet,37.565960,126.980522,4c67b557b80abe9a37cad0e5,81,Casual
18,광화문국밥,Gukbap Restaurant,37.569020,126.975582,58d48a6ada54ae0db6b5a19e,74,Korean


In [ ]:
pd.crosstab(index = raw_dataset['categories_classified'],
            columns = 'count')

col_0,count
categories_classified,
American,5
Asian,13
Bar,5
Casual,22
Europe,3
Korean,14


In [ ]:
# First: where to bin the data

print(np.percentile(raw_dataset['likes'], 33))
print(np.percentile(raw_dataset['likes'], 66))

17.260000000000005
51.820000000000036


In [ ]:
# create a function to bin for us

def rankings(s):
  if s['likes'] <= 17:
    return 3
  elif s['likes'] <= 51:
    return 2
  else:
    return 1

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,likes,categories_classified,ranking
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192,Korean,1
2,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41,Korean,2
3,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57,Casual,1
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46,Korean,2
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13,Casual,3
10,Sushi Cho (스시 조),Sushi Restaurant,37.564491,126.979743,4f82efddd4f208b5bd478f0e,114,Asian,1
14,Wolhyang (월향),Pub,37.567916,126.975665,566033b1498e93c84b700d64,50,Bar,2
16,열빈,Chinese Restaurant,37.567522,126.976187,4f969714e4b07e54f0d8cdb9,16,Asian,3
17,La Seine (라세느),Buffet,37.565960,126.980522,4c67b557b80abe9a37cad0e5,81,Casual,1
18,광화문국밥,Gukbap Restaurant,37.569020,126.975582,58d48a6ada54ae0db6b5a19e,74,Korean,1


#### 2.3 Exploratory Data Analysis

##### The Greatest & Worst resturant in Seoul

In [ ]:
# show the greatest restaurant (for these likes) list
first = raw_dataset[raw_dataset['ranking'] == 1].sort_values(by = 'likes', ascending=False)
first.head(1)

,name,categories,lat,lng,id,likes,categories_classified,ranking
65,만족오향족발,Bossam/Jokbal Restaurant,37.563408,126.975884,4ba33ce1f964a520363138e3,222,Korean,1


In [ ]:
# show the worst restaurant (for these likes) list
last = raw_dataset[raw_dataset['ranking'] == 3].sort_values(by = 'likes', ascending=True)
last.head(1)

,name,categories,lat,lng,id,likes,categories_classified,ranking
66,비어할레 / Bier Halle (비어할레),Beer Garden,37.567572,126.981408,4bee9c46e8c3c92840db9892,7,Bar,3


##### The Ratios for features in the dataset

In [ ]:
count_class = pd.crosstab(index = raw_dataset['categories_classified'],
            columns = 'count')

In [ ]:
count_class.index

Index(['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean'], dtype='object', name='categories_classified')

In [ ]:
# the ratio of categories_classified

labels = count_class.index
values = count_class['count']

fig = go.Figure()
fig.add_trace(go.Pie(labels=labels,
                     values=values,
                     hole=.3))

fig.update_traces(textposition='inside',
                  textinfo='percent+label')

fig.show()

In [ ]:
count_class2 = pd.crosstab(index = raw_dataset['categories'],
            columns = 'count')

In [ ]:
count_class2

col_0,count
categories,
BBQ Joint,2
Bagel Shop,1
Bakery,1
Beer Garden,1
Bistro,2
Bossam/Jokbal Restaurant,2
Buffet,2
Burger Joint,1
Café,4


In [ ]:
# the ratio of categories

labels = count_class2.index
values = count_class2['count']

fig = go.Figure()
fig.add_trace(go.Pie(labels=labels,
                     values=values,
                     hole=.3))

fig.update_traces(textposition='inside',
                  textinfo='percent+label')

fig.show()

In [ ]:
# looking for the names of coffee shop / cafe in Seoul
raw_dataset[(raw_dataset['categories'] == 'Coffee Shop') | (raw_dataset['categories'] == 'Café')]

,name,categories,lat,lng,id,likes,categories_classified,ranking
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13,Casual,3
27,Café MAMAS (카페마마스),Café,37.567216,126.979188,4b84c56bf964a520a34331e3,79,Casual,1
33,Paul Bassett (폴바셋),Coffee Shop,37.568019,126.976677,5146a5ece4b088751f847bee,139,Casual,1
46,Paul Bassett (폴바셋),Coffee Shop,37.570404,126.978832,55efabb5498ef918a753f0f2,46,Casual,2
49,Starbucks Reserve (스타벅스 리저브),Coffee Shop,37.564463,126.979025,4b5bb69af964a520a31129e3,137,Casual,1
59,CONFECTIONS BY FOUR SEASONS,Coffee Shop,37.570462,126.975299,56175cb7498e144b13689c4b,20,Casual,2
61,Starbucks (스타벅스),Coffee Shop,37.564000,126.978832,54e4032b498ea9b63904a623,15,Casual,3
64,수수커피,Café,37.571723,126.979010,59afb2389de23b1bcc2de6d8,16,Casual,3
83,Starbucks (스타벅스),Coffee Shop,37.570267,126.978990,55e24a57498e6b62f70697e0,58,Casual,1
90,Starbucks Reserve (스타벅스 리저브),Coffee Shop,37.563005,126.974359,5456b3f7498eccfb5b024079,47,Casual,2


What a Starbucks!

#### 2.4 Exploring Data in the map of Seoul

In [ ]:
raw_dataset.columns

Index(['name', 'categories', 'lat', 'lng', 'id', 'likes',
       'categories_classified', 'ranking'],
      dtype='object')

In [ ]:
! pip install git+https://github.com/python-visualization/branca.git@master 
# 한글 깨짐 해결을 위해 필요

  Cloning https://github.com/python-visualization/branca.git (to revision master) to /tmp/pip-req-build-m_6v06ft
  Running command git clone -q https://github.com/python-visualization/branca.git /tmp/pip-req-build-m_6v06ft
  Created wheel for branca: filename=branca-0.4.1+3.g5887b9b-cp36-none-any.whl size=24523 sha256=2e5466a9c8f95e30759b32d70f9eef3fa940c490bdc27963aa2c46bc6dd4d2b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-q336wxye/wheels/14/bc/41/d6078abe73beea93d98c014bba54c395224cabb54f95b9d248
Successfully built branca


In [ ]:
import folium
map_seoul = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, name, categories_classified in zip(raw_dataset['lat'],
                                           raw_dataset['lng'],
                                           raw_dataset['name'],
                                           raw_dataset['categories_classified']):
  label = '{}, {}'.format(name, categories_classified)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius = 4,
      popup = label,
      color = 'blue',
      fill = True,
      fill_color = '#87cefa',
      fill_opacity = 0.5,
      parse_html = False).add_to(map_seoul)

map_seoul

### 3. Methodology

This project will utilize both linear and logistic regression machine learning methods to train and test the data. Namely, linear regression will be used in an attempt to predict the number of "likes" a new restaurant in this region will have. We will utilize the Sci-Kit Learn Package to run the model.

We can also utilize logistic regression as a classification method rather than direct prediction of the number of likes. Since the number of "likes" can be binned into different categories based on different percentile bins, it is also potentiallly possible to see which range of "likes" a new restaurant in this region will have.

Since the "likes" are binned into multiple (more than 2) categories, the type of logistic regression will be multinomial. Additionally, although the ranges are indeed discrete categories, they are also ordinal in nature. Therefore the logistic regression will need to be specified as being both multinomial and ordinal. This can be done through the Sci-Kit Learn Package as well.

In [ ]:
# create dummies for linear regression modeling
reg_dataset = pd.DataFrame()

# one-hot encoding
reg_dataset = pd.get_dummies(raw_dataset['categories_classified'],
                             prefix="", prefix_sep="")

# add name, ranking, and likes columns back to dataframe
# 등수가 낮을 수록 좋은게 맞지만 regression에서는 정반대가 필요
reg_dataset['ranking'] = raw_dataset['ranking'] * -1
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]

reg_dataset.head()

,name,American,Asian,Bar,Casual,Europe,Korean,ranking,likes
0,무교동북어국집,0,0,0,0,0,1,-1,192
2,Joo Ok (주옥),0,0,0,0,0,1,-2,41
3,Läderach chocolatier suisse (레더라),0,0,0,1,0,0,-1,57
6,철철복집,0,0,0,0,0,1,-2,46
8,Be-Up Coffee (비읍커피),0,0,0,1,0,0,-3,13


### 4. Result

#### 4.1 Linear Regression

In [ ]:
# multiple linear regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y = np.asanyarray(train['likes'])
regr.fit(x, y)

# the coefficients
print('Coefficients: ', regr.coef_)

Coefficients:  [-24.52013889   6.97986111  12.57986111   5.16736111 -19.52013889
  19.31319444]


In [ ]:
# multiple linear regression prediction capabilities

y_hat = regr.predict(test[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
x = np.asanyarray(test[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y = np.asanyarray(test['likes'])

print('Residual sum of squares: %.2f' % np.mean((y_hat - y) ** 2))

# explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 2887.91
Variance score: -0.05


#### 4.2 Logistic Regression Results

A multinomial ordinal logisitc regression model was trained on a random subsample of 80% of the sample and then tested on the other 20%. To see if this is a reasonable model, its jaccard similarity score and log-loss were calculated (66.66% and 1.009 respectively). Although this is not a perfect prediction, a similarity of 66% between the training set and test set is a reasonable result. The classification report is also printed later on below. 

Given the modestly accurate ability of this model, we can also run the model on the full dataset. The coefficients show that opening a restaurant in Seoul, opening a bar, or serving cuisine that is american or asian in nature, are associated negatively with "likes."

In [ ]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y_test = np.asanyarray(test['ranking'])


# LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
# LR

mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[-0.1232046   0.37697094 -0.32188974  0.18497265  0.16961722 -0.28646513]


In [ ]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob

jaccard_similarity_score(y_test, yhat)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:664: FutureWarning:

jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.



0.3333333333333333

In [ ]:
log_loss(y_test, yhat_prob)

1.1319823409693552

In [ ]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y_all = np.asanyarray(reg_dataset['ranking'])

LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print (coef)

[-0.13603284  0.32301168 -0.25933572  0.11947425  0.06039165 -0.10750886]


In [ ]:
print (classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -3       0.25      0.75      0.38         4
          -2       0.67      0.33      0.44         6
          -1       0.00      0.00      0.00         5

    accuracy                           0.33        15
   macro avg       0.31      0.36      0.27        15
weighted avg       0.33      0.33      0.28        15



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

